# Homework 3: Interactive Viz

In [1]:
import branca.colormap as cm
import folium
import glob
import json
import matplotlib.pyplot as plt
import os
import pandas as pd
import seaborn as sns

from IPython.core.display import display, HTML
from ipywidgets import widgets
from ipywidgets import interact

import warnings
warnings.filterwarnings('ignore')

In [2]:
PATH = '/Users/Gaurav/Desktop/Courses/Applied Data Analysis/ADA2017-Homeworks/03 - Interactive Viz/topojson/'
# PATH = '/Users/karunya/Documents/EPFL/ADA/ADA2017-Homeworks/03 - Interactive Viz/topojson/'
# PATH = '/Users/laurieprelot/Documents/GitHub/ADA2017-Homeworks/03 - Interactive Viz/'

DATA_EURO = PATH + 'europe.topojson.json'
DATA_SWISS = PATH + 'ch-cantons.topojson.json'

DATA_FOLDER = '/Users/Gaurav/Desktop/'
# DATA_FOLDER = '/Users/karunya/Documents/EPFL/ADA/ADA_Group_Tota_Pasari_Prelot/'
# DATA_FOLDER = '/Users/laurieprelot/Documents/GitHub/ADA2017-Homeworks/03 - Interactive Viz/Eurostats /'

In [3]:
with open(DATA_EURO) as euro_file:    
    euro_data = json.load(euro_file)
    
with open(DATA_SWISS) as swiss_file:    
    swiss_data = json.load(swiss_file)

In this homework you will be dealing with two different datasets from the statistics offices of the European commission (eurostat) and the Swiss Confederation (amstat). They provide a variety of datasets with plenty of information on many different statistics and demographics at their respective scales. Unfortunately, as is often the case is data analysis, these websites are not always straightforward to navigate. They may include a lot of obscure categories, not always be translated into your native language, have strange link structures, … Navigating this complexity is part of a data scientists' job : you will have to use a few tricks to get the right data for this homework.

For the visualization part, install Folium (HINT: it is not available in your standard Anaconda environment, therefore search on the Web how to install it easily!). Folium's README comes with very clear examples, and links to their own iPython Notebooks -- make good use of this information. For your own convenience, in this same directory you can already find two .topojson files, containing the geo-coordinates of

European countries (liberal definition of EU) (topojson/europe.topojson.json, source)
Swiss cantons (topojson/ch-cantons.topojson.json)
These will be used as an overlay on the Folium maps.

### 1. Go to the eurostat website and try to find a dataset that includes the european unemployment rates at a recent date.

Website: http://ec.europa.eu/eurostat/data/database

Use this data to build a Choropleth map which shows the unemployment rate in Europe at a country level. Think about the colors you use, how you decided to split the intervals into data classes or which interactions you could add in order to make the visualization intuitive and expressive. Compare Switzerland's unemployment rate to that of the rest of Europe.

### Preprocessing of the data frame from Eurostats 

First we create one table with all the informations from the website 

In [4]:
filenames = glob.glob(DATA_FOLDER + "Eurostats/*.csv")
if len(filenames) != 27:
    print("Files missing. Check path again!")

EuroData = pd.read_csv(filenames[0])
for i in range(1, len(filenames)):
    EuroData = pd.concat([EuroData, pd.read_csv(filenames[i])], axis=0)

In [5]:
EuroData.drop(['Flag and Footnotes'], axis=1, inplace=True)
EuroData.head()

,TIME,GEO,AGE,UNIT,SEX,Value
0,2007,European Union (28 countries),Total,Percentage of active population,Females,7.9
1,2008,European Union (28 countries),Total,Percentage of active population,Females,7.5
2,2009,European Union (28 countries),Total,Percentage of active population,Females,8.9
3,2010,European Union (28 countries),Total,Percentage of active population,Females,9.6
4,2011,European Union (28 countries),Total,Percentage of active population,Females,9.8


In [6]:
# range of values for each category
print('Age range is ', EuroData.AGE.unique())
print('Time range is', EuroData.TIME.unique())
print('Metric is ', EuroData.UNIT.unique())
print('Sex values are', EuroData.SEX.unique())

Age range is  ['Total' 'Less than 25 years' 'From 25 to 74 years']
Time range is [2007 2008 2009 2010 2011 2012 2013 2014 2015 2016]
Metric is  ['Percentage of active population' 'Percentage of total population'
 'Thousand persons']
Sex values are ['Females' 'Males' 'Total']


In [7]:
# select the countries of interest
EuroData = EuroData.set_index(['TIME','SEX', 'AGE', 'UNIT'])

countries = ['Belgium',
             'Bulgaria',
             'Czech Republic',
             'Denmark',
             'Germany (until 1990 former territory of the FRG)',
             'Estonia',
             'Ireland',
             'Greece',
             'Spain',
             'France',
             'Croatia',
             'Italy',
             'Cyprus',
             'Latvia',
             'Lithuania',
             'Luxembourg',
             'Hungary',
             'Malta',
             'Netherlands',
             'Austria',
             'Poland',
             'Portugal',
             'Romania',
             'Slovenia',
             'Slovakia',
             'Finland',
             'Sweden',
             'United Kingdom',
             'Iceland',
             'Norway']

EuroData = EuroData[EuroData['GEO'].isin(countries)]

In [8]:
# add a correction for Germany name 
EuroData['GEO'][EuroData['GEO'].isin(['Germany (until 1990 former territory of the FRG)'])] = 'Germany'

In [9]:
# correct data type by removing commas and converting to float
print(EuroData['Value'].dtype)
EuroData['Value'] = (EuroData['Value']).str.replace(',', '.')
EuroData['Value'] = (EuroData['Value']).astype('float')
print(EuroData['Value'].dtype)

object
float64


In [10]:
EuroData.head()

,,,,GEO,Value
TIME,SEX,AGE,UNIT,,
2007,Females,Total,Percentage of active population,Belgium,8.5
2008,Females,Total,Percentage of active population,Belgium,7.6
2009,Females,Total,Percentage of active population,Belgium,8.1
2010,Females,Total,Percentage of active population,Belgium,8.5
2011,Females,Total,Percentage of active population,Belgium,7.2


In [11]:
EuroData.dropna(axis=0, how='any', inplace=True)

#### We focus on the following:

1. Metrics:
'Percentage of total population' because the metric 'Percentage of active population' has too many NA values.
 
2. Age ranges:
We ignore 'Total' and use 'Less than 25 years' and 'From 25 to 74 years' to create different layers for the map.

3. Sex types:
Both 'Male' and 'Female'.

Note: We choose not to separate over the years and show the data for the year 2016.

In [12]:
# extract data based on values of the time, sex, age, and unit columns
unemployment_all_ages = EuroData.loc[2016, 'Total', 'Total', 'Percentage of total population']
unemployment_youth = EuroData.loc[2016, 'Total', 'Less than 25 years', 'Percentage of total population']
unemployment_adults = EuroData.loc[2016, 'Total', 'From 25 to 74 years', 'Percentage of total population']
unemployment_females_all_ages = EuroData.loc[2016, 'Females', 'Total', 'Percentage of total population']
unemployment_males_all_ages = EuroData.loc[2016, 'Males', 'Total', 'Percentage of total population']

# create a list for the chloropeths below
maps = [unemployment_all_ages,
        unemployment_youth,
        unemployment_adults,
        unemployment_females_all_ages,
        unemployment_males_all_ages]

# add legends for each map
legends = ['Unemployment rate in total population and all ages',
           'Unemployment rate in less than 25 years old',
           'Unemployment rate in from 25 to 74 years old',
           'Unemployment rate in female population of all ages',
           'Unemployment rate in male population of all ages']

In [13]:
# we create a chloropeth with unemployment rate for all ages in european countries
total_unemployment_rate = folium.Map([47.87083, 9.08572], tiles='Mapbox Bright', zoom_start=4)

total_unemployment_rate.choropleth(euro_data, 
             name=legends[0],
             data=maps[0], 
             columns=['GEO','Value'],
             key_on='feature.properties.NAME',
             fill_color='YlOrRd',
             topojson='objects.europe',
             line_color='gray',
             legend_name=legends[0],
             threshold_scale=list(maps[0].Value.quantile([0, .15, .25, .5, .9, 1.]).values))

folium.LayerControl().add_to(total_unemployment_rate)
total_unemployment_rate.save('Total_Unemployment_Rate.html')
total_unemployment_rate

### NOTE: In order to check each layer, click on the small gray and white icon with layers on the right. By default, all four layers are displayed simultaneously.

In [14]:
# we create a multilayer chloropeth using four of our data categories defined in legends
multilayered_unemployment_rate = folium.Map([47.87083, 9.08572], tiles='Mapbox Bright', zoom_start=4)

for i in range (1, len(maps)):
    multilayered_unemployment_rate.choropleth(euro_data, 
                 name=legends[i],
                 data=maps[i], 
                 columns=['GEO','Value'],
                 key_on='feature.properties.NAME',
                 fill_color='YlOrRd',
                 topojson='objects.europe',
                 line_color='gray',
                 legend_name=legends[i],
                 threshold_scale=list(maps[i].Value.quantile([0, .15, .25, .5, .9, 1.]).values))

folium.LayerControl().add_to(multilayered_unemployment_rate)
multilayered_unemployment_rate.save('Multilayered_Unemployment_Rate.html')
multilayered_unemployment_rate

# NOTE: In order to check each layer, click on the small gray and white icon with layers on the right
# by default, all four layers are displayed simultaneously

## 2. Go to the amstat website to find a dataset that includes the unemployment rates in Switzerland at a recent date.

Website: https://www.amstat.ch/v2/index.jsp

HINT Go to the details tab to find the raw data you need. If you do not speak French, German or Italian, think of using free translation services to navigate your way through.
Use this data to build another Choropleth map, this time showing the unemployment rate at the level of swiss cantons. Again, try to make the map as expressive as possible, and comment on the trends you observe.

The Swiss Confederation defines the rates you have just plotted as the number of people looking for a job divided by the size of the active population (scaled by 100). This is surely a valid choice, but as we discussed one could argue for a different categorization.

Copy the map you have just created, but this time don't count in your statistics people who already have a job and are looking for a new one. How do your observations change? You can repeat this with different choices of categories to see how selecting different metrics can lead to different interpretations of the same data.

In [ ]:
df = pd.read_excel(DATA_FOLDER + 'Unemployment_Cantons.xlsx', encoding='encoding', index=False)

# remove uneccessary rows and columns
df = df.iloc[1:,:]
df = df.drop('Monat', 1)
df = df[['Kanton', 'Gesamt']]

# rename columns
df.columns = ['Canton', 'Unemployment Rate']
df.head()

In [ ]:
# convert dtype for column from object to float for calculations
print(df['Unemployment Rate'].dtype)
df['Unemployment Rate'] = (df['Unemployment Rate']).str.replace(',', '.')
df['Unemployment Rate'] = (df['Unemployment Rate']).astype('float')
print(df['Unemployment Rate'].dtype)

In [ ]:
ch_unemployment_rate = folium.Map([46.8182, 8.2275], tiles='Mapbox Bright', zoom_start=7.5)

ch_unemployment_rate.choropleth(
    geo_data=swiss_data, 
    name='Unemployment Rate in Canton',
    data=df, 
    columns=['Canton', 'Unemployment Rate'],
    fill_color='OrRd',
    topojson='objects.cantons',
    key_on='feature.properties.name',
    legend_name='Unemployment Rate in Canton',
    threshold_scale=list(df['Unemployment Rate'].quantile([0, .15, .25, .5, .9, 1.]).values))

folium.LayerControl().add_to(ch_unemployment_rate)
ch_unemployment_rate.save('Unemployment_Rate_Canton.html')
ch_unemployment_rate

### 3. Use the amstat website again to find a dataset that includes the unemployment rates in Switzerland at recent date, this time making a distinction between Swiss and foreign workers.

Website: https://www.amstat.ch/v2/index.jsp

The Economic Secretary (SECO) releases a monthly report on the state of the employment market. In the latest report (September 2017), it is noted that there is a discrepancy between the unemployment rates for foreign (5.1%) and Swiss (2.2%) workers.

Show the difference in unemployment rates between the two categories in each canton on a Choropleth map (hint The easy way is to show two separate maps, but can you think of something better?). Where are the differences most visible? Why do you think that is?

Now let's refine the analysis by adding the differences between age groups. As you may have guessed it is nearly impossible to plot so many variables on a map. Make a bar plot, which is a better suited visualization tool for this type of multivariate data.

## Unemployment difference based on Nationality

In [ ]:
# read file from given location
filenames = glob.glob(DATA_FOLDER + "Unemployment_Canton_Nationality.xlsx")

# if exactly one file is not found, print error
if len(filenames) != 1:
    print("File missing. Check path again!")

In [ ]:
# remove first two rows, set correct header, and re-index
df = pd.read_excel(filenames[0])
df.columns = df.iloc[1]
df = df.iloc[3:-1]
df = df.reset_index(drop=True)

# rename column names
df.columns.values[0] = "Canton"
df.columns.values[1] = "Nationality"
df.columns.values[-2] = "Total Rate"

# select only the required columns
columns_needed = ["Canton", "Nationality", "Total Rate"]
df = df[columns_needed]

print(df.shape)
df.head()

In [ ]:
# convert dtype for column from object to float for calculations
print(df["Total Rate"].dtype)
df["Total Rate"] = (df["Total Rate"]).astype("float")
print(df["Total Rate"].dtype)

In [ ]:
# range of values for each category
print("Total Rate ranges from {} to {}".format(df["Total Rate"].min(), df["Total Rate"].max()))
print("Nationality has the following unique values: ", df.Nationality.unique())
print("Canton has the following unique values: ", df.Canton.unique())

In [ ]:
# create dataframes based on nationality - swiss or foreign
df_swiss = df.loc[df.Nationality == "Schweizer"]
df_foreign = df.loc[df.Nationality == "Ausländer"]

In [ ]:
# re-index new dataframe
df_swiss = df_swiss.reset_index(drop=True)
df_swiss.head()

In [ ]:
# re-index new dataframe
df_foreign = df_foreign.reset_index(drop=True)
df_foreign.head()

In [ ]:
# copy the foreign unemployment rate dataframe
difference_map = df_foreign.copy()

# rename columns
difference_map.columns.values[-1] = "Difference"
difference_map["Nationality"] = "NA"

# calculate difference in unemployment rate by foreign minus swiss
difference_map["Difference"] = df_foreign["Total Rate"] - df_swiss["Total Rate"]

# set index
difference_map = difference_map.set_index(["Nationality"])

print(difference_map.columns)

difference_map

In [ ]:
# we create a chloropeth with unemployment rate for all ages in european countries
difference_unemployment_rate_nationality = folium.Map([46.8, 8.25], tiles='Mapbox Bright', zoom_start=8)

legend = "Difference in unemployment rate = (Foreign - Swiss) workers"

difference_unemployment_rate_nationality.choropleth(swiss_data, 
             name=legend,
             data=difference_map, 
             columns=['Canton','Difference'],
             key_on='feature.properties.name',
             fill_color='YlOrRd',
             topojson='objects.cantons',
             line_color='gray',
             legend_name=legend,
             threshold_scale=list(difference_map["Difference"].quantile([0, .15, .25, .5, .9, 1.]).values))

folium.LayerControl().add_to(difference_unemployment_rate_nationality)
difference_unemployment_rate_nationality.save('Difference_Unemployment_Rate_Nationality.html')
difference_unemployment_rate_nationality

### Observations:

We see that the biggest differences between foreign workers being unemployed as compared to swiss workers is in the cantons of Jura and Valais. These cantons have higher number of foreign workers and are not near the major universities or cities such as Geneva, Lausanne or Zurich, which otherwise would have given more jobs. There are also cultural differences in these regions which can be a shock for foreign workers.

## Unemployment difference based on Age Groups

In [ ]:
# read file from given location
filenames = glob.glob(DATA_FOLDER + "Unemployment_Canton_Age_Groups.xlsx")

# if exactly one file is not found, print error
if len(filenames) != 1:
    print("File missing. Check path again!")

In [ ]:
# remove first two rows, set correct header, and re-index
df = pd.read_excel(filenames[0])
df.columns = df.iloc[1]
df = df.iloc[3:-1]
df = df.reset_index(drop=True)

# rename column names
df.columns.values[0] = "Canton"
df.columns.values[1] = "Age Group"
df.columns.values[-2] = "Total Rate"

# select only the required columns
columns_needed = ["Canton", "Age Group", "Total Rate"]
df = df[columns_needed]

print(df.shape)
df.head()

In [ ]:
# convert dtype for column from object to float for calculations
print(df["Total Rate"].dtype)
df["Total Rate"] = (df["Total Rate"]).astype("float")
print(df["Total Rate"].dtype)

In [ ]:
# range of values for each category
print("Total Rate ranges from {} to {}".format(df["Total Rate"].min(), df["Total Rate"].max()))
print("Age Group has the following unique values: ", df["Age Group"].unique())
print("Canton has the following unique values: ", df.Canton.unique())

In [ ]:
# index by age groups on Y axis and cantons on X axis
df_age_groups = df.reset_index().pivot(index="Age Group", columns="Canton", values="Total Rate")
df_age_groups

In [ ]:
# for better visualization, divide table into two halves
df_age_groups1 = df_age_groups.iloc[0:5]
df_age_groups2 = df_age_groups.iloc[5:,]

In [ ]:
# plot bar graphs
df_age_groups1.plot.bar(figsize=(18, 10))

plt.title("Unemployment Rate by Age Groups for all Swiss Cantons", fontsize=15)
plt.xlabel("Total Unemployment Rate")
plt.ylabel("Age Groups")
plt.show()

In [ ]:
df_age_groups2.plot.bar(figsize=(18, 10))

plt.title("Unemployment Rate by Age Groups for all Swiss Cantons", fontsize=15)
plt.xlabel("Total Unemployment Rate")
plt.ylabel("Age Groups")
plt.show()

### 4. BONUS: using the map you have just built, and the geographical information contained in it, could you give a rough estimate of the difference in unemployment rates between the areas divided by the Röstigraben?